## What does Canada produce locally?

In [28]:
# import visual libraries and read data
import panel as pn
pn.extension('tabulator', template='material')
import pandas as pd
import hvplot.pandas
import holoviews as hv
import colorcet as cc

dfGroupProd = pd.read_pickle('Q1data/GroupProduction.pkl')
Production = pd.read_pickle('Q1data/CanadaProduction.pkl')
dfPyramidPreds = pd.read_pickle('Q1data/dfPyramidPreds.pkl')
dfCanadianIntake = pd.read_pickle('Q1data/dfCanadianIntake.pkl')
dfAvail = pd.read_pickle('Q1data/dfAvail.pkl')
dfImports = pd.read_csv('Q1data/dfImportsViz.csv')
dfExports = pd.read_csv('Q1data/dfExportsViz.csv')

In [18]:
# create 2 linked widgets
dfGroupProd.drop('index', axis=1, inplace=True)
interactiveGroupProd = dfGroupProd.interactive(sizing_mode='stretch_both')
selectProdYear = pn.widgets.IntRangeSlider(name='Year', start=int(dfGroupProd['Year'].unique()[4]), end=int(dfGroupProd['Year'].unique()[0]))
sliderProdValue = pn.widgets.IntSlider(name='Number of Items', start=0, end=dfGroupProd['Item'].unique().size, value=dfGroupProd['Item'].unique().size)

filtered = interactiveGroupProd[(interactiveGroupProd['Year']>=selectProdYear.value_start) & (interactiveGroupProd['Year']<=selectProdYear.value_end)] 
filtered2 = filtered.head(sliderProdValue.value)

In [12]:
# create two bar plots, add interactivity to view top items produced
widgetGroupProduction = filtered2.groupby(['Group', 'Value', 'Year']).sum().reset_index().hvplot(x='Group', y='Value', kind='barh', title='Amount Produced Per Food Group (2015 - 2019)', 
    min_height=1500, min_width=1500)
widgetItemProduction = filtered2.groupby(['Item', 'Value', 'Year']).sum().reset_index().hvplot(x='Item', y='Value', kind='barh', title='Amount Produced per Item (2015 - 2019)',
    min_height=1500, min_width=1500)
widgetUnitProduction = filtered2.groupby(['Unit', 'Value', 'Year']).sum().reset_index().hvplot(x='Unit', y='Value', kind='barh', title='Amount Produced per Unit of Measure', 
    min_height=1500, min_width=1500)
tabProd = pn.Tabs(min_height=1500, min_width=1500, dynamic=True)
widgetboxProd = pn.WidgetBox(selectProdYear, sliderProdValue, margin=5)
tabProd.extend([
    ('Group', widgetGroupProduction.holoviews()),
    ('Item', widgetItemProduction.holoviews()),
    ('Unit', widgetUnitProduction.holoviews())])

In [13]:
panelProdTitle = pn.panel('## Candian Food Production (2015-2019)')
panelProd = pn.Column(panelProdTitle, tabProd)

## How balanced is the Canadian diet? What parts of the diet could be improved?

In [37]:
interactiveServing = dfCanadianIntake.interactive(sizing_mode='stretch_both')
selectServingGroup = pn.widgets.Select(name='Food Group', options=[i for i in dfCanadianIntake.Group.unique()], size=6)
filterServingGroup = interactiveServing[interactiveServing['Group'] == selectServingGroup.value]

In [40]:
widgetAge = filterServingGroup.groupby(['Age Range', 'serving', 'year', 'Group']).sum().reset_index().hvplot(
    kind='barh', x='Age Range', y='serving', title='Age vs Serving', min_height=1500, min_width=1500, colorbar=True, c='year', cmap=cc.glasbey_light)
widgetEducation = filterServingGroup.groupby(['Education Level', 'serving', 'year', 'Group']).sum().reset_index().hvplot(
    kind='barh', x='Education Level', y='serving', title='Eduction vs Serving', min_height=1500, min_width=1500, colorbar=True, c='year', cmap=cc.glasbey_light)
widgetLocality = filterServingGroup.groupby(['Locality', 'serving', 'year', 'Group']).sum().reset_index().hvplot(
    kind='bar', x='Locality', y='serving', title='Locality vs Serving', min_height=1500, min_width=1500, colorbar=True, c='year', cmap=cc.glasbey_light)
widgetGender = filterServingGroup.groupby(['Gender', 'serving', 'year', 'Group']).sum().reset_index().hvplot(
    kind='bar', x='Gender', y='serving', title='Gender vs Serving', min_height=1500, min_width=1500, colorbar=True, c='year', cmap=cc.glasbey_light)

tabServing = pn.Tabs(min_height=1500, min_width=1500, dynamic=True)
tabServing.extend([
    ('Age', widgetAge.holoviews()),
    ('Education', widgetEducation.holoviews()),
    ('Locality', widgetLocality.holoviews()),
    ('Gender', widgetGender.holoviews()),
])

In [44]:
widgetServings = dfPyramidPreds.hvplot(x="year", groupby='Group')
titleServing = pn.panel('''
---
## Canadian serving intake per food group (1990 - 2018)''')
panelServing = pn.Column(titleServing, selectServingGroup, tabServing, widgetServings)

BokehModel(combine_events=True, render_bundle={'docs_json': {'6fdb6401-2ec6-4e31-a2b8-29bd63c6abc2': {'defs': …

Column
    [0] Markdown(str)
    [1] Select(name='Food Group', options=['Fruits Group', ...], size=6, value='Fruits Group')
    [2] Tabs
        [0] HoloViews(DynamicMap, name='Age')
        [1] HoloViews(DynamicMap, name='Education')
        [2] HoloViews(DynamicMap, name='Locality')
        [3] HoloViews(DynamicMap, name='Gender')
    [3] Row
        [0] HoloViews(DynamicMap)
        [1] Column
            [0] WidgetBox
                [0] Select(margin=(20, 20, 20, 20), name='Group', options=['Beans, Eggs, ...], value='Beans, Eggs, ..., width=250)
            [1] VSpacer()

## Food Availability

In [29]:
interactiveAvail = dfAvail.interactive(sizing_mode='stretch_both')
selectAvailSupply = pn.widgets.Select(name='Supply/Disposition', options=[i for i in dfAvail['Supply and disposition'].unique()], size=10)
filterAvailSupply = interactiveAvail[interactiveAvail['Supply and disposition'] == selectAvailSupply.value]

In [36]:
widgetAvailGroup = filterAvailSupply.groupby(['Group', 'VALUE', 'Supply and disposition']).sum().reset_index()
widgetAvailGroup = widgetAvailGroup.hvplot(kind='barh', x='Group', y='VALUE', responsive=True, min_height=1500, min_width=1500)
widgetAvailDate = filterAvailSupply.hvplot(kind='box', x='Group', y='VALUE', min_height=1500, min_width=1500, responsive=True, c='REF_DATE', cmap=cc.glasbey_dark)
widgetAvailSupply = dfAvail.groupby(['Supply and disposition', 'VALUE']).sum().reset_index()
widgetAvailSupply = widgetAvailSupply.hvplot(kind='barh', x='Supply and disposition', y='VALUE', responsive=True, min_height=1500, min_width=1500)

lsAvail = hv.link_selections.instance(unselected_alpha=0.02)
linkAvailWidgets = (lsAvail(widgetAvailGroup) + lsAvail(widgetAvailDate) + lsAvail(widgetAvailSupply)).cols(1).opts(
    hv.opts.Histogram(tools=['box_select', 'lasso_select', 'tap', 'hover']),
    hv.opts.Layout(shared_datasource=True, toolbar='left'))

tableCommodity = filterAvailSupply.pipe(lsAvail.filter, selection_expr=lsAvail.param.selection_expr)[['Commodity', 'Group', 'REF_DATE', 'VALUE']].groupby('Commodity').sum().reset_index().pipe(
    pn.widgets.Tabulator, pagination='remote', page_size=15)
panelAvailTitle = pn.panel('''
---
## Total Supply and disposition of Food in Canada (1990 - 2020)''')
panelAvail = pn.Column(panelAvailTitle, selectAvailSupply, linkAvailWidgets, tableCommodity)

In [ ]:
pn.state.template.sidebar_width = 350
pn.state.template.title = 'DAAN Capstone Project Q1'
panelQ1 = pn.Column(
    panelProd, panelServing, panelAvail
)

In [ ]:
(widgetboxProd).servable(area='sidebar')

In [ ]:
panelQ1.servable(title="# How much of Canada’s population can be fed on food grown nationally? Which countries should we work with to fill gaps in the food supply?​")